#### Program that pulls counts and geometric information from a given point in a Jupiter cube.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import interp1d
import math
from uncertainties import ufloat
from uncertainties.umath import *

#### Programs from Pat Irwin, translated from IDL

In [ ]:
def interceptellip(a,b,alpha,beta,gamma,x0,y0,z0):
    '''
    ; **********************************************************
    ; procedure to find the intercepts (if any) between the line
    ;
    ;  (x-x0)       (y-y0)      (z-z0)
    ;  ------  =    ------   =  ------
    ;  alpha         beta       gamma
    ; 
    ; and the ellipsoid
    ;
    ;
    ;  x^2    y^2    z^2   
    ;  --- +  --- +  ---  = 1
    ;  a^2    a^2    b^2
    ;
    ; Input variables
    ;       a       real    ellipsoid semi-major axis
    ;       b       real    ellipsoid semi-minor axis
    ;       alpha   real    line x-gradient
    ;       beta    real    line y-gradient
    ;       gamma   real    line z-gradient
    ;       x0      real    line x-intercept
    ;       y0      real    line y-intercept
    ;       z0      real    line z-intercept
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if line intercepts, set to -1  otherwise
    ;       x(2)    real    x-intercepts
    ;       y(2)    real    y-intercepts
    ;       z(2)    real    z-intercepts
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3-8-19
    ; **********************************************************
    '''
    a1 = 1.0/a**2 + (beta/(a*alpha))**2 + (gamma/(b*alpha))**2

    b1 = (-2*x0*beta**2/alpha**2 + 2*beta*y0/alpha)/a**2
    b1 = b1 + (-2*x0*gamma**2/alpha**2 + 2*gamma*z0/alpha)/b**2
    
    c1 = ((beta*x0/alpha)**2 - 2*beta*y0*x0/alpha + y0**2)/a**2
    c1 = c1 + ((gamma*x0/alpha)**2 - 2*gamma*x0*z0/alpha + z0**2)/b**2 -1
        
    #;print,a1,1.0/a**2 + (gamma/(b*alpha))**2
    #;print,b1,2*gamma*z0/(alpha*b**2)
    #;print,c1,(y0/a)**2 + (z0/b)**2 -1
    
    xtest = b1**2 - 4*a1*c1

    x = np.zeros(2)
    y = np.zeros(2)
    z = np.zeros(2)
        
    if xtest > 0.0:
        iflag = 1
        x[0] = (-b1 + np.sqrt(xtest))/(2*a1)
        x[1] = (-b1 - np.sqrt(xtest))/(2*a1)
        y[0] = y0 + (beta/alpha)*(x[0]-x0)
        y[1] = y0 + (beta/alpha)*(x[1]-x0)
        z[0] = z0 + (gamma/alpha)*(x[0]-x0)
        z[1] = z0 + (gamma/alpha)*(x[1]-x0)

        # testing to see if solution is on ellipsoid
        test=np.ndarray(len(x))
        for i in range(0,len(x)):
            test[i] = (x[i]/a)**2 + (y[i]/a)**2 + (z[i]/b)**2
        xtest1 = abs(test[0]-1.0)
        xtest2 = abs(test[1]-1.0)
        
        err = 1e-5
        
        if xtest1 > err or xtest2 > err:
            print 'Problem in interceptellip - solution not on ellipsoid'
            print 'Test =',test

    return iflag,x,y,z

In [ ]:
def projposolar(Re,obl,epsilon,latsol,lonsol,se_lon,eoff,poff):
    '''
    ; ************************************************************
    ; Procedure to find latitude and longitude and zenith angle of
    ; intercept between line and ellipsoid
    ;
    ; Input variables
    ;       Re      real    Equatorial Radius (arcsec)
    ;       obl     real    Planetary oblateness
    ;       epsilon real    Sub-observer (planetocentric) latitude
    ;       latsol  real    Sub-solar planetocentric latitude
    ;       lonsol  real    longitude difference between sub-solar and sub-observer
    ;                       points.
    ;       se_lon  real    Sub-observer longitude # added by Emma
    ;       eoff    real    equatorial offset of beam (arcsec)
    ;       poff    real    polar offset of beam (arcsec)
    ;
    ; Output variables
    ;       iflag   integer Set to 1 if real intercept, -1 otherwise
    ;       xlat    real    Planetocentric latitude
    ;       longitude real  xlon+se_lon, offset of longitude added to sub-observer longitude # added by Emma
    ;       xlon    real    Longitude
    ;       zen     real    Zenith angle
    ;       szen    real    Solar zenith angle
    ;       aphi    real    local azimuth angle between sun and observer
    ;
    ; Pat Irwin     11/2/07
    ; Python conversion - Emma Dahl 3/8/19
    ; ************************************************************
    '''
    
    dtr = np.pi/180.0 # radians/degrees
    Rp = Re*(1.0-obl)
    
    x0 = 0.0
    y0 = eoff
    z0 = poff/np.cos(epsilon*dtr)
    
    alpha = np.sin(np.pi/2.0 - epsilon*dtr)
    beta = 0.0
    gamma = np.cos(np.pi/2.0 - epsilon*dtr)
        
    iflag,x,y,z = interceptellip(Re,Rp,alpha,beta,gamma,x0,y0,z0)
    
    xlat = 0.0
    xlon = 0.0
    zen = 0.0
    
    if iflag > 0:
        # if real intercept, find lat, long, and zenith
        
        # find distance along line of sight
        lambdaa = (x-x0)/alpha
        if lambdaa[0] > lambdaa[1]:
            inear = 0
        else:
            inear = 1
                
        x1 = x[inear]
        y1 = y[inear]
        z1 = z[inear]
        
        r = np.sqrt(x1**2 + y1**2 + z1**2)
        
        theta = np.arccos(z1/r)
        xlat = 90.0 - theta/dtr
                
        #; convert to planetographic latitude
        #; xlat = np.arctan(((Re/Rp)**2)*np.tan(xlat*dtr))/dtr
        
        cphi = x1/(r*np.sin(theta))
        
        if cphi > 1.0:
            cphi = 1.0
        if cphi < -1.0:
            cphi = -1.0
                    
        phi = np.arccos(cphi)
        if y1 < 0.0:
            phi = -phi
        xlon = phi/dtr        
        
        # Finding aphi, zen, szen - don't mess with these, want to still output them
        
        v1 = np.zeros(3)
        v2 = np.zeros(3)
        v3 = np.zeros(3)
        
        # v1 is normal vector of point observed
        v1[0] = x1/r
        v1[1] = y1/r
        v1[2] = z1/r
        
        v2[0] = alpha
        v2[1] = beta
        v2[2] = gamma
        
        summ = 0.0
        
        for i in range(0,3):
            summ += v1[i]*v2[i]
        zen = np.arccos(summ)/dtr
        
        # Finding aphi
        
        alphasol = np.sin(np.pi/2.0 - latsol*dtr)*np.cos(lonsol*dtr)
        betasol =  np.sin(np.pi/2.0 - latsol*dtr)*np.sin(lonsol*dtr)
        gammasol = np.cos(np.pi/2 - latsol*dtr)
        v3[0]=alphasol
        v3[1]=betasol
        v3[2]=gammasol

        summ = 0.0
        for i in range(0,3):
            summ += v1[i]*v3[i]
        szen = np.arccos(summ)/dtr
        
        cphase = 0.0
        
        for i in range(0,3):
            cphase += v2[i]*v3[i]
            
        a = np.cos(zen*dtr)*np.cos(szen*dtr)
        b = np.sin(zen*dtr)*np.sin(szen*dtr)
        
        if b == 0.0:
            aphi = 180.0
        else:
            cphi = (cphase-a)/b
            aphi = 180.0-np.arccos(cphi)/dtr
            
    longitude = se_lon-xlon # offset of longitude added to sub-observer longitude
    # Remember that system III longitude increases to the west
    
    #return xlat,longitude
    return iflag,xlat,longitude,xlon,zen,szen,aphi

#### Step 1: Collect geometric information from images that have been made into a cube

In [ ]:
# STEP 1: 
# prepare to run projposolar on all images by accumulating required info. 

cube = fits.open('/Users/dahlek/Desktop/march2017_cube_redo_2.fits')

# data directory, list
directory = '/Volumes/external_hd/march2017/redo/jupiter/'
imlist = np.loadtxt('/Volumes/external_hd/march2017/'+'march_2017_sciimages',dtype=str)

# DEFINE PLATE SCALE HERE

#platescale = 0.08479 #"/pix - post-2018 camera
platescale = 0.10593 #"/pix - pre-2018 camera

# values from horizons - not dependent on observation
eq_rad_km = 71492 # jupiter's equatorial radius in km
oblateness = 0.06487 # from wikipedia

# make functions for finding planetocentric latitude from planetographic latitude
# file from http://w.astro.berkeley.edu/~mikewong/doc/kmdeg_0.25.txt
coordinates = np.loadtxt('/Users/dahlek/Desktop/jup_geo_coordinates',dtype=float)

graph = [] # planetographic coordinates
centric = [] # planetocentric coordinates

wl = []

for i in range(0,len(coordinates)):
    graph.append(coordinates[i][0])
    centric.append(coordinates[i][1])
    
# centric as a function of graph
centric_func = interp1d(graph,centric)

k = 0

# array of info. wl (nm), equatorial radius (arcsec), oblateness, epsilon, latsol, lonsol, se_lon
geo_info = np.zeros((cube[0].header['naxis3'],7))

# Find inputs from Jupiter data that has already been run through disk_corr. (need planet center!)
for i in range(0,len(imlist)):
    im = fits.open(directory+imlist[i])
    if im[0].header['rfon'] == 1:
        print imlist[i],im[0].header['lambda'],'nm'
        geo_info[k,0] = im[0].header['lambda']
        eq_rad_arcsec = round(np.arctan(eq_rad_km/im[0].header['PE_RNG'])*206265,4)
        print 'Re',eq_rad_arcsec
        geo_info[k,1] = eq_rad_arcsec
        print 'obl:',oblateness
        geo_info[k,2] = oblateness
        if im[0].header['SE_LAT'] < 0:
            print 'epsilon:', -centric_func(abs(im[0].header['SE_LAT'])) # degrees
            geo_info[k,3] = -centric_func(abs(im[0].header['SE_LAT']))
        else:
            print 'epsilon:', centric_func(im[0].header['SE_LAT']) # degrees
            geo_info[k,3] = centric_func(abs(im[0].header['SE_LAT']))
        if im[0].header['SS_LAT'] < 0:
            print 'latsol:', -centric_func(abs(im[0].header['SS_LAT'])) # degrees
            geo_info[k,4] = -centric_func(abs(im[0].header['SS_LAT']))
        else:
            print 'latsol:', centric_func(im[0].header['SS_LAT']) # degrees
            geo_info[k,4] = centric_func(abs(im[0].header['SS_LAT']))
        print 'lonsol:',im[0].header['SS_LON']-im[0].header['SE_LON']
        geo_info[k,5] = im[0].header['SS_LON']-im[0].header['SE_LON']
        print 'se_lon:',im[0].header['SE_LON']
        geo_info[k,6] = im[0].header['SE_LON']
        k+=1 # a check to make sure list of info and cube are the same length        
        
        print ' '
    im.close()
    
if k != cube[0].header['naxis3']:
    print 'ERROR! Unequal numbers of images in list for geometric info and in image cube.'

#### Step 2: Look at image in cube, use pixel position to find the latitude/longitude of a particular feature you want to analyze via trial and error. Most plots here are just diagnostic. Will use the equatorial and polar offset pixel values to define the chosen lat and long that the spectrum will be extracted from.

In [ ]:
# STEP 2:
cube = fits.open('/Users/dahlek/Desktop/march2017_cube_redo.fits')
image_index = 240 # Change this value to see how a position moves over the course of the cube, but be sure to define your final pixel positions based on the 0th image, bc of the loop organizaiton in Step 4. 

# half a length of the box of averaged pixels (eg box_size=5 will result in a 10x10 box):
box_size = 5 # where you define your pixel box for the rest of the program. carries over into step 4.

plt.matshow(cube[0].data[image_index],origin='lower')

# plot center
plt.plot(int(int(len(cube[0].data[1])/2)),int(int(len(cube[0].data[1])/2)),'bo')

# Some examples:

#NEB
#equatorial_offset_pix = -19 # moves point E-W
#polar_offset_pix = 59 # moves point N-S

#EZ
equatorial_offset_pix = -20 # moves point E-W
polar_offset_pix = 0 # moves point N-S

#SEB
#equatorial_offset_pix = -19 # moves point E-W
#polar_offset_pix = -35 # moves point N-S

# testing longitudes
#equatorial_offset_pix = -180 # moves point E-W
#polar_offset_pix = 0 # moves point N-S

# south tropical zone, center
#equatorial_offset_pix =  -19
#polar_offset_pix = -80

# south tropical zone, limb
#equatorial_offset_pix =  150
#polar_offset_pix = -80


# plot center of box
plt.plot(int(int(len(cube[0].data[1])/2))+equatorial_offset_pix,\
         int(int(len(cube[0].data[1])/2))+polar_offset_pix,\
         'ro')

# make a mask around the chosen lat/long
mask = np.zeros((len(cube[0].data[1]),len(cube[0].data[1])))
mask[(int(int(len(cube[0].data[1])/2))+polar_offset_pix-box_size):\
     (int(int(len(cube[0].data[1])/2))+polar_offset_pix+box_size),\
    (int(int(len(cube[0].data[1])/2))+equatorial_offset_pix-box_size):\
    (int(int(len(cube[0].data[1])/2))+equatorial_offset_pix+box_size)] = 1

# make the square visible on the cube
cube[0].data[image_index][(int(int(len(cube[0].data[1])/2))+polar_offset_pix-box_size):\
     (int(int(len(cube[0].data[1])/2))+polar_offset_pix+box_size),\
    (int(int(len(cube[0].data[1])/2))+equatorial_offset_pix-box_size):\
    (int(int(len(cube[0].data[1])/2))+equatorial_offset_pix+box_size)] = 1

plt.matshow(cube[0].data[image_index],origin='lower')

#plt.matshow(mask,origin='lower')

plt.show()

#### Step 3:  Convert the pixel offset to arcseconds, find the corresponding latitude and longitude. This will be the lat and long that is most closely followed.

In [ ]:
# convert those values to offset in arcseconds

f = 0 #the index of the image you used to find the pixel offsets in Step 2; should be 0 in both cases.

equatorial_offset_arcsec = equatorial_offset_pix*platescale
polar_offset_arcsec = polar_offset_pix*platescale

chosen_lat = projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6],equatorial_offset_arcsec,polar_offset_arcsec)[1]

chosen_long = projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6],equatorial_offset_arcsec,polar_offset_arcsec)[2]

print 'sub-observer longitude:',geo_info[f,6]
print 'equatorial, polar offset (in arcseoncds):',equatorial_offset_arcsec,polar_offset_arcsec
print 'latitude and longitude:', chosen_lat,' & ', chosen_long

#### Step 4: Using initial pixel offset and chosen latitude and longitude, loop through cube and find the closest pixel offset to chosen latitude and longitude in each frame.

In [ ]:
cube = fits.open('/Users/dahlek/Desktop/march2017_cube_redo.fits')

# initial values (pixel will be updated over loop)
long_offset_pixel = np.copy(polar_offset_pix) # Value from earlier. moves point E-W
lat_offset_pixel = np.copy(equatorial_offset_pix) # Value from earlier. moves point N-S

print chosen_lat,chosen_long

spec = [] # list of spectral points
poisson_error = [] # list of error (derived from poisson noise) for each averaged square
# Geometric info:
latitude_final_list = [] # !! planetocentric
longitude_final_list = []
xlon_list= []
zen_list= []
szen_list= []
aphi_list= []

for f in range(0,cube[0].header['naxis3']):
    print ' '
    print geo_info[f,0], f # wavelength
    # lists of pixel values to test, plus or minus 5:
    long_offset_pixels_list = []
    lat_offset_pixels_list = []
    
    for i in range(-6,6):
        long_offset_pixels_list.append(long_offset_pixel+i)
        lat_offset_pixels_list.append(lat_offset_pixel+i)

    # convert to arcseconds
    lat_offset_arcseconds_list = []
    long_offset_arcseconds_list = []
    for i in range(0,len(long_offset_pixels_list)):
        long_offset_arcseconds_list.append(long_offset_pixels_list[i]*platescale)
    for i in range(0,len(lat_offset_pixels_list)):
        lat_offset_arcseconds_list.append(lat_offset_pixels_list[i]*platescale)

    #print lat_offset_arcseconds_list
    #print long_offset_arcseconds_list

    # make functions for the geometry of this image:
    difference_tuples = [] # list of differences
    pixel_tuples = [] # list of corresponding pixel positions
    distances = [] # distances of differences, this is what needs to be minimized
    
    for i in range(0,len(lat_offset_arcseconds_list)):
        for j in range(0,len(long_offset_arcseconds_list)):

            pixel_tuples.append([lat_offset_pixels_list[i],long_offset_pixels_list[j]])
            difference_tuples.append([chosen_lat-projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcseconds_list[i], long_offset_arcseconds_list[j])[1],\
                                      chosen_long-projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcseconds_list[i], long_offset_arcseconds_list[j])[2]])
            distances.append(np.sqrt((chosen_lat-projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcseconds_list[i], long_offset_arcseconds_list[j])[1])**2 + ( chosen_long-projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcseconds_list[i], long_offset_arcseconds_list[j])[2])**2))
            
    #print distances
    # Find the pixel offset that provides the minimum distance from the chosen latitude and longitude.  
    val, idx = min((val, idx) for (idx, val) in enumerate(distances))
    #print 'minimum distance',val
    #print 'index of minimum',idx
    print 'pixel position',pixel_tuples[idx]
    
    # x and y pixel positions:
    g = int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]
    h = int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]
    
    # Find the error for the average of the square
    # loop through each pixel in the square, define the error assocaited with that value
    # N +- sqrt(N)
    ddd = [] # dummy list for error values
    for p in range((int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]-box_size),\
     (int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]+box_size)):
        for l in range((int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]-box_size),\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]+box_size)):
            ddd.append(ufloat(cube[0].data[f][p][l],np.sqrt(cube[0].data[f][p][l])))
    # reformat ddd
    '''spec_and_error = np.zeros((len(ddd),2))
    for z in range(0,len(ddd)):
        spec_and_error[z,0] = float('{:10.2f}'.format(ddd[z])[:10])
        spec_and_error[z,1] = float('{:10.2f}'.format(ddd[z])[13:])
    print spec_and_error'''
    poisson_error.append(float('{:10.2f}'.format(np.mean(ddd))[13:]))
    
    # append the average of the square
    spec.append(np.mean(cube[0].data[f][(int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]-box_size):\
     (int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]+box_size),\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]-box_size):\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]+box_size)]))
    
    # make square visible, plot
    cube[0].data[f][(int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]-box_size):\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][1]+box_size),\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]-box_size):\
    (int(len(cube[0].data[1])/2)+pixel_tuples[idx][0]+box_size)] = 1
    #plt.matshow(cube[0].data[f],origin='lower')
    #plt.plot(int(len(cube[0].data[1])/2)+pixel_tuples[idx][0],int(len(cube[0].data[1])/2)+pixel_tuples[idx][1],'r.')
    #plt.show()
    
    # update initial pixel position for next image
    long_offset_pixel = pixel_tuples[idx][1]
    lat_offset_pixel = pixel_tuples[idx][0]
    
    #print lat_offset_pixel,long_offset_pixel
    # convert offset pixel to offset in arcseconds to plug into projposolar
    long_offset_arcsec = long_offset_pixel*platescale
    lat_offset_arcsec = lat_offset_pixel*platescale
    
    # run projposolar on that position
    iflag,latitude_final,longitude_final,xlon,zen,szen,aphi = projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcsec, long_offset_arcsec)
    latitude_final_list.append(latitude_final)
    longitude_final_list.append(longitude_final)
    xlon_list.append(xlon)
    zen_list.append(zen)
    szen_list.append(szen)
    aphi_list.append(aphi)
    
    print 'latitude:',projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcsec, long_offset_arcsec)[1]
    print 'longitude:',projposolar(geo_info[f,1],geo_info[f,2],geo_info[f,3],geo_info[f,4],geo_info[f,5],geo_info[f,6], lat_offset_arcsec, long_offset_arcsec)[2]

#### Plot stuff

In [ ]:
plt.plot(geo_info[:,0],spec)
#plt.errorbar(geo_info[:,0],spec,yerr=poisson_error)
plt.title('Latitude: '+str(chosen_lat)+'  Longitude: '+str(chosen_long))
plt.show()

In [ ]:
plt.plot(geo_info[:,0],aphi_list)
plt.title('local azimuth angle between sun and observer')
plt.ylabel('aphi')
plt.xlabel('wavelength/time')
plt.show()

In [ ]:
plt.plot(geo_info[:,0],zen_list)
plt.title('Zenith angle (emiss_ang)')
plt.ylabel('zen')
plt.xlabel('wavelength/time')
plt.show()

In [ ]:
# cosine of emission zenith angle. want it to be 1 near the center of the disk, near 0.2 for something close to the limb.
plt.plot(geo_info[:,0],np.cos(np.array(zen_list)*(2*math.pi/360.)))
plt.title('Cosine of zenith angle')
plt.ylabel('zen')
plt.xlabel('wavelength/time')
plt.show()

In [ ]:
plt.plot(geo_info[:,0],szen_list)
plt.title('Solar zenith angle')
plt.ylabel('szen')
plt.xlabel('wavelength/time')
plt.show()

In [ ]:
plt.plot(geo_info[:,0],latitude_final_list)
plt.xlabel('wavelength/time')
plt.ylabel('Latitude (Planetocentric, deg)')
plt.show()

In [ ]:
plt.plot(geo_info[:,0],longitude_final_list)
plt.xlabel('wavelength/time')
plt.ylabel('Longitude (Sys III, deg)')
plt.show()

#### Last step - save info to text files

In [ ]:
# use this file to make .spx file
dummy = np.zeros((cube[0].header['naxis3'],9))

In [ ]:
dummy[:,0] = np.linspace(470,950,241) #geo_info[:,0] # wavelength
dummy[:,1] = spec # spectrum
dummy[:,2] = latitude_final_list
dummy[:,3] = longitude_final_list
dummy[:,4] = xlon_list
dummy[:,5] = zen_list
dummy[:,6] = szen_list
dummy[:,7] = aphi_list
dummy[:,8] = poisson_error
# Comment to add manually to the beginning of the file:
print '# chosen_lat='+str(chosen_lat)+' chosen_long='+str(chosen_long)
print '# initial pixel offsets: equatorial_offset_pix='+str(equatorial_offset_pix)+' polar_offset_pix='+str(polar_offset_pix)+' averaged box size='+str(2*box_size)
print '# wavelength(nm) | DN/sec | latitude | longitude | xlon | zenith angle | solar zenith angle | azimuth angle |  poisson error (counts/sec)'

In [ ]:
# Change file name to lat and long
feature = 'South_trop_zone_limb'
np.savetxt('/Users/dahlek/Desktop/nemesis_inputs_march2019/'+str(feature)+'_averaged',dummy)

#### Bonus code for making images of Jupiter w/ locations of spectra boxes:

In [ ]:
# making image for paper of locations of spectra

cube = fits.open('/Users/dahlek/Desktop/march2017_cube.fits')

cube[0].data[117][(int(len(cube[0].data[1])/2)+59-box_size):\
     (int(len(cube[0].data[1])/2)+59+box_size),\
    (int(len(cube[0].data[1])/2)+1-box_size):\
    (int(len(cube[0].data[1])/2)+1+box_size)] = 0

cube[0].data[117][(int(len(cube[0].data[1])/2)+0-box_size):\
     (int(len(cube[0].data[1])/2)+0+box_size),\
    (int(len(cube[0].data[1])/2)-1-box_size):\
    (int(len(cube[0].data[1])/2)-1+box_size)] = 0

cube[0].data[117][(int(len(cube[0].data[1])/2)-35-box_size):\
     (int(len(cube[0].data[1])/2)-35+box_size),\
    (int(len(cube[0].data[1])/2)-1-box_size):\
    (int(len(cube[0].data[1])/2)-1+box_size)] = 0


plt.matshow(cube[0].data[117],origin='lower',cmap='gray')

#plt.plot(int(len(cube[0].data[1])/2)+2,int(len(cube[0].data[1])/2)+56,'r.') # NEB aat 712 nm

#plt.plot(int(len(cube[0].data[1])/2)-1,int(len(cube[0].data[1])/2)+3,'r.') # EZ at 712 nm

#plt.plot(int(len(cube[0].data[1])/2)-0,int(len(cube[0].data[1])/2)+-35,'r.') # SEB at 712 nm

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labeltop=False) # labels along the bottom edge are off
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title('710 nm - spectra locations')
plt.show()

In [ ]:
plt.matshow(cube[0].data[25],origin='lower',cmap='gray')
plt.plot(int(len(cube[0].data[1])/2)+20,int(len(cube[0].data[1])/2)+55,'r.') # NEB aat 700 nm
plt.plot(int(len(cube[0].data[1])/2)+19,int(len(cube[0].data[1])/2)+7,'r.') # EZ at 700 nm
plt.plot(int(len(cube[0].data[1])/2)+20,int(len(cube[0].data[1])/2)+-33,'r.') # SEB at 700 nm
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labeltop=False) # labels along the bottom edge are off
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
plt.title('526 nm')
plt.show()